# Title
[]()

In [4]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
import requests
import json

In [2]:
datetime.timestamp(datetime.strptime('2023-02-01', "%Y-%m-%d"))

1675238400.0

In [8]:
datetime.strptime('2023-02-01', "%Y-%m-%d") < datetime.strptime('2023-02-21', "%Y-%m-%d")

True

In [5]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['ig_user_id']
access_token = credentials['access_token']

In [10]:
from datetime import time, datetime
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
        url_without_token += f'&since={datetime.timestamp(since)}'
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        url_without_token += f'&until={datetime.timestamp(until)}'


    url = url_without_token+'&access_token='+access_token
    
    print(url_without_token)
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while earliest_end_time > since:
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(json_normalize(response_json_dict[page], record_path='data'))
    #     except:
    #         print('No data in request response for page', page)
    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['next']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    # try:
    #     df = pd.concat(df_list)
    #     print('Number of posts:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         save_csv(df,filename,csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    # return df, response_json_dict

get_ig_account_insights(ig_user_id, access_token, since='2022-02-24', until='2023-02-24')

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&since=1645689600.0&until=1677225600.0


# *End of Page*